In [3]:
import pandas as pd
import numpy as np
import pymannkendall as mk
import os
from typing import Dict, List, Tuple
import glob

# ======================
# DATA PROCESSING MODULE
# ======================

def load_holaps_data(file_path: str) -> pd.DataFrame:
    """
    Load HOLAPS model data and prepare for analysis
    
    Args:
        file_path: Path to HOLAPS CSV file
    
    Returns:
        Processed DataFrame with additional columns
    """
    df = pd.read_csv(file_path)
    
    # Convert temperature from Kelvin to Celsius if needed
    temp_cols = ['Ta', 'Td', 'Ts']
    for col in temp_cols:
        if col in df.columns:
            # Assuming temperatures are in Kelvin, convert to Celsius
            df[col] = df[col] - 273.15
    
    # Create datetime column for easier time-based operations
    df['date'] = pd.to_datetime(df[['year', 'month']].assign(day=1))
    df['Year'] = df['year']
    df['Month'] = df['month']
    
    return df

# ======================
# COMPLETENESS ANALYSIS
# ======================

def validate_holaps_variable(df: pd.DataFrame, var: str) -> List[int]:
    """
    Identify complete years for a HOLAPS variable
    
    Args:
        df: Input DataFrame with data
        var: Variable name to validate
    
    Returns:
        List of years with complete data (12 months)
    """
    # HOLAPS data typically doesn't have QC flags, just check for non-NaN values
    valid_mask = df[var].notna()
    
    valid_data = df[valid_mask].copy()
    valid_year_counts = valid_data['Year'].value_counts()
    return valid_year_counts[valid_year_counts == 12].index.tolist()

def get_complete_years_holaps(df: pd.DataFrame, variables: List[str]) -> List[int]:
    """
    Find years where all specified variables have complete data
    
    Args:
        df: Input DataFrame with data
        variables: List of variables to check
    
    Returns:
        Sorted list of years with complete data for all variables
    """
    common_years = set(df['Year'].unique())
    for var in variables:
        valid_years = validate_holaps_variable(df, var)
        common_years.intersection_update(valid_years)
    return sorted(common_years)

def save_holaps_variable_availability(df: pd.DataFrame, variables: List[str], output_file: str) -> None:
    """
    Save availability of each variable (years with valid data)
    
    Args:
        df: DataFrame with Year and variable columns
        variables: List of variables to check
        output_file: Path to save CSV
    """
    availability_records = []
    
    for var in variables:
        valid_years = validate_holaps_variable(df, var)
        availability_records.append({
            "Variable": var,
            "Valid_Years": ", ".join(map(str, sorted(valid_years))),
            "Count_Valid_Years": len(valid_years)
        })
    
    availability_df = pd.DataFrame(availability_records)
    availability_df.to_csv(output_file, index=False)
    print(f"HOLAPS variable availability saved to {output_file}")

# ======================
# YEARLY ANALYSIS
# ======================

def calculate_holaps_yearly_means(df: pd.DataFrame, variables: List[str], years: List[int]) -> pd.DataFrame:
    """
    Calculate yearly means for complete years only
    
    Args:
        df: Input DataFrame with data
        variables: Variables to calculate means for
        years: Years to include
    
    Returns:
        DataFrame with yearly means
    """
    if not years:
        return pd.DataFrame()
    
    complete_data = df[df['Year'].isin(years)]
    yearly_means = complete_data.groupby('Year')[variables].mean()
    return yearly_means.reset_index()

def generate_holaps_yearly_analysis(df: pd.DataFrame, variables: List[str]) -> Dict[str, pd.DataFrame]:
    """
    Generate yearly means for HOLAPS data
    
    Args:
        df: Input DataFrame with data
        variables: List of variables to analyze
    
    Returns:
        Dictionary of DataFrames with yearly means
    """
    results = {}
    
    # Get complete years for all variables
    complete_years = get_complete_years_holaps(df, variables)
    
    if complete_years:
        # Calculate yearly means for complete period
        yearly_means = calculate_holaps_yearly_means(df, variables, complete_years)
        results["Complete Period"] = yearly_means
        
        # Also calculate for all available years (individual variable completeness)
        for var in variables:
            var_years = validate_holaps_variable(df, [var])
            if var_years:
                var_means = calculate_holaps_yearly_means(df, [var], var_years)
                results[f"Single Variable - {var}"] = var_means
    
    return results

# ======================
# SEASONAL ANALYSIS
# ======================

def get_season_months(season: str) -> List[int]:
    """Return months for each season (with DJF year adjustment)"""
    return {
        'DJF': [12, 1, 2],  # December, January, February
        'MAM': [3, 4, 5],   # March, April, May
        'JJA': [6, 7, 8],   # June, July, August
        'SON': [9, 10, 11]  # September, October, November
    }[season]

def get_complete_seasons_holaps(df: pd.DataFrame, variables: List[str], season: str) -> List[int]:
    """
    Get seasons with complete data for all variables
    
    Args:
        df: Input DataFrame
        variables: Variables to check
        season: Season code (DJF, MAM, JJA, SON)
    
    Returns:
        List of complete season years
    """
    season_months = get_season_months(season)
    df_season = df[df['Month'].isin(season_months)].copy()
    
    # Adjust year for winter season (DJF)
    if season == 'DJF':
        df_season['Season_Year'] = df_season['Year']
        df_season.loc[df_season['Month'] == 12, 'Season_Year'] += 1
    else:
        df_season['Season_Year'] = df_season['Year']
    
    # Find seasons with all months present
    month_counts = df_season.groupby(['Season_Year'])['Month'].nunique()
    complete_seasons = month_counts[month_counts == 3].index.tolist()
    
    # Check variable completeness
    valid_seasons = []
    for season_year in complete_seasons:
        season_data = df_season[df_season['Season_Year'] == season_year]
        valid = True
        for var in variables:
            var_data = season_data[season_data[var].notna()]
            if len(var_data) < 3:  # Not all months valid
                valid = False
                break
        if valid:
            valid_seasons.append(season_year)
    
    return sorted(valid_seasons)

def calculate_holaps_seasonal_means(df: pd.DataFrame, variables: List[str], season: str, season_years: List[int]) -> pd.DataFrame:
    """
    Calculate seasonal means for complete seasons
    
    Args:
        df: Input DataFrame
        variables: Variables to calculate means for
        season: Season code (DJF, MAM, JJA, SON)
        season_years: List of years with complete seasonal data
    
    Returns:
        DataFrame with seasonal means
    """
    if not season_years:
        return pd.DataFrame()
    
    season_months = get_season_months(season)
    df_season = df[df['Month'].isin(season_months)].copy()
    
    # Adjust year for winter season (DJF)
    if season == 'DJF':
        df_season['Season_Year'] = df_season['Year']
        df_season.loc[df_season['Month'] == 12, 'Season_Year'] += 1
    else:
        df_season['Season_Year'] = df_season['Year']
    
    # Filter for complete seasons
    df_season = df_season[df_season['Season_Year'].isin(season_years)]
    
    # Calculate seasonal means
    seasonal_means = df_season.groupby('Season_Year')[variables].mean()
    seasonal_means['Season'] = season
    return seasonal_means.reset_index()

def generate_holaps_seasonal_analysis(df: pd.DataFrame, variables: List[str], season: str) -> Dict[str, pd.DataFrame]:
    """
    Generate seasonal means for HOLAPS data
    
    Args:
        df: Input DataFrame
        variables: Variables to analyze
        season: Season code (DJF, MAM, JJA, SON)
    
    Returns:
        Dictionary of DataFrames with seasonal means
    """
    results = {}
    
    # Get complete seasons for all variables
    complete_seasons = get_complete_seasons_holaps(df, variables, season)
    
    if complete_seasons:
        # Calculate seasonal means for complete period
        seasonal_means = calculate_holaps_seasonal_means(df, variables, season, complete_seasons)
        results["Complete Period"] = seasonal_means
        
        # Also calculate for all available seasons (individual variable completeness)
        for var in variables:
            # For single variable, we can be less strict about completeness
            var_season_data = df[df['Month'].isin(get_season_months(season))].copy()
            
            # Adjust year for winter season
            if season == 'DJF':
                var_season_data['Season_Year'] = var_season_data['Year']
                var_season_data.loc[var_season_data['Month'] == 12, 'Season_Year'] += 1
            else:
                var_season_data['Season_Year'] = var_season_data['Year']
            
            # Filter for seasons with this variable available
            var_valid = var_season_data[var_season_data[var].notna()]
            var_season_counts = var_valid.groupby('Season_Year').size()
            var_complete_seasons = var_season_counts[var_season_counts == 3].index.tolist()
            
            if var_complete_seasons:
                var_means = calculate_holaps_seasonal_means(df, [var], season, var_complete_seasons)
                results[f"Single Variable - {var}"] = var_means
    
    return results

# ======================
# TREND ANALYSIS
# ======================

def calculate_holaps_trend_stats(series: pd.Series) -> Dict[str, float]:
    """
    Calculate Mann-Kendall trend statistics for HOLAPS data
    
    Args:
        series: Time series data with Year as index
    
    Returns:
        Dictionary with slope, p-value, and period info
    """
    if series.notna().sum() < 10:
        return None
    
    start_year = int(series.index.min())
    end_year = int(series.index.max())
    full_index = np.arange(start_year, end_year + 1, 1)
    series = series.reindex(full_index)
    
    try:
        result = mk.original_test(series.values)
        return {
            'slope': result.slope,
            'p_value': result.p,
            'period': f"{start_year}-{end_year} ({series.notna().sum()})"
        }
    except Exception:
        return None

def build_holaps_trend_tables(sheets: Dict[str, pd.DataFrame]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Build p-value and slope tables from HOLAPS analysis results
    
    Args:
        sheets: Dictionary of DataFrames from analysis
    
    Returns:
        Tuple of (p-value table, slope table)
    """
    p_table_data = []
    slope_table_data = []
    
    for sheet_name, data in sheets.items():
        if data.empty:
            continue
            
        # Set appropriate index
        if 'Year' in data.columns:
            data_indexed = data.set_index('Year')
        elif 'Season_Year' in data.columns:
            data_indexed = data.set_index('Season_Year')
        else:
            continue
        
        # Get variables (exclude metadata columns)
        vars_included = [col for col in data_indexed.columns if col not in ['Season', 'station_id', 'latitude', 'longitude']]
        
        # Calculate stats for all variables
        stats_dict = {}
        period = None
        
        for var in vars_included:
            stats = calculate_holaps_trend_stats(data_indexed[var].dropna())
            if stats:
                stats_dict[var] = {
                    'p_value': stats['p_value'],
                    'slope': stats['slope']
                }
                period = stats['period']
        
        if stats_dict:
            p_row = {'Analysis': sheet_name, 'Period': period}
            s_row = {'Analysis': sheet_name, 'Period': period}
            
            for var in vars_included:
                if var in stats_dict:
                    p_row[var] = stats_dict[var]['p_value']
                    s_row[var] = stats_dict[var]['slope']
                else:
                    p_row[var] = np.nan
                    s_row[var] = np.nan
            
            p_table_data.append(p_row)
            slope_table_data.append(s_row)
    
    return pd.DataFrame(p_table_data), pd.DataFrame(slope_table_data)

# ======================
# MAIN EXECUTION
# ======================

def run_holaps_analysis(station_code: str):
    """
    Run complete HOLAPS analysis pipeline
    
    Args:
        station_code: Station code (e.g., "FR-Hes")
    """
    # Define HOLAPS variables
    holaps_variables = ['GHF', 'H', 'LE', 'Rn', 'Ta', 'Td', 'Ts', 'rain', 'sm1']
    
    # Build file path
    base_dir = r"c:\\Deepak\\HOLAPS\\monthly\\station_data"
   
    holaps_file = os.path.join(base_dir, f"{station_code}_all_variables.csv")
    
    # Check if file exists
    if not os.path.exists(holaps_file):
        print(f"HOLAPS file not found for {station_code}: {holaps_file}")
        return
    
    # Create output directory
    output_dir = os.path.join(base_dir, station_code)
    os.makedirs(output_dir, exist_ok=True)
    
    # Load data
    print(f"Loading HOLAPS data for {station_code}...")
    df = load_holaps_data(holaps_file)
    
    # Basic info
    print(f"Data period: {df['Year'].min()} to {df['Year'].max()}")
    print(f"Total years: {df['Year'].nunique()}")
    
    # Save variable availability
    availability_file = os.path.join(output_dir, f"{station_code}_holaps_variable_availability.csv")
    save_holaps_variable_availability(df, holaps_variables, availability_file)
    
    # Yearly analysis
    print("Performing yearly analysis...")
    yearly_results = generate_holaps_yearly_analysis(df, holaps_variables)
    
    yearly_file = os.path.join(output_dir, f'holaps_yearly_means_{station_code}.xlsx')
    if yearly_results:
        with pd.ExcelWriter(yearly_file) as writer:
            for sheet_name, data in yearly_results.items():
                if not data.empty:
                    data.to_excel(writer, sheet_name=sheet_name[:31], index=False)
        print(f"Yearly means saved to {yearly_file}")
    else:
        print("No complete yearly data available")
    
    # Seasonal analysis
    print("Performing seasonal analysis...")
    seasons = ['DJF', 'MAM', 'JJA', 'SON']
    
    for season in seasons:
        seasonal_results = generate_holaps_seasonal_analysis(df, holaps_variables, season)
        seasonal_file = os.path.join(output_dir, f"holaps_seasonal_means_{station_code}_{season}.xlsx")
        
        if seasonal_results:
            with pd.ExcelWriter(seasonal_file) as writer:
                for sheet_name, data in seasonal_results.items():
                    if not data.empty:
                        data.to_excel(writer, sheet_name=sheet_name[:31], index=False)
            print(f"Seasonal means for {season} saved to {seasonal_file}")
        else:
            print(f"No complete seasonal data available for {season}")
    
    # Trend analysis - Yearly
    print("Performing trend analysis...")
    if os.path.exists(yearly_file):
        yearly_sheets = pd.read_excel(yearly_file, sheet_name=None)
        p_table, slope_table = build_holaps_trend_tables(yearly_sheets)
        
        trend_file = os.path.join(output_dir, f'holaps_trend_tables_{station_code}_yearly.xlsx')
        with pd.ExcelWriter(trend_file) as writer:
            p_table.to_excel(writer, sheet_name='P-values', index=False)
            slope_table.to_excel(writer, sheet_name='Slopes', index=False)
        print(f"Yearly trend tables saved to {trend_file}")
    
    # Trend analysis - Seasonal
    for season in seasons:
        seasonal_file = os.path.join(output_dir, f"holaps_seasonal_means_{station_code}_{season}.xlsx")
        if os.path.exists(seasonal_file):
            try:
                seasonal_sheets = pd.read_excel(seasonal_file, sheet_name=None)
                p_table, slope_table = build_holaps_trend_tables(seasonal_sheets)
                
                trend_file = os.path.join(output_dir, f'holaps_trend_tables_{station_code}_{season}.xlsx')
                with pd.ExcelWriter(trend_file) as writer:
                    p_table.to_excel(writer, sheet_name='P-values', index=False)
                    slope_table.to_excel(writer, sheet_name='Slopes', index=False)
                print(f"Seasonal trend tables for {season} saved to {trend_file}")
            except Exception as e:
                print(f"Error processing seasonal trends for {season}: {e}")

def run_all_holaps_stations():
    base_dir = r"c:\\Deepak\\HOLAPS\\monthly\\station_data"
    
    # Changed pattern to find all_variables files
    holaps_files = glob.glob(os.path.join(base_dir, "*_all_variables.csv"))
    
    # Extract station codes from filenames like "BE-Bra_all_variables.csv"
    stations = []
    for file_path in holaps_files:
        filename = os.path.basename(file_path)
        station_code = filename.replace("_all_variables.csv", "")
        stations.append(station_code)
    
    print(f"Found {len(stations)} HOLAPS stations: {stations}")
    
    # Run analysis for each station
    for station in stations:
        print(f"\n{'='*50}")
        print(f"Processing HOLAPS station: {station}")
        print(f"{'='*50}")
        
        try:
            run_holaps_analysis(station)
            print(f"✓ Completed analysis for {station}")
        except Exception as e:
            print(f"✗ Error processing {station}: {e}")

# Run for specific station
if __name__ == "__main__":
    # Run for a single station
    # run_holaps_analysis("FR-Hes")
    
    # Or run for all available stations
    run_all_holaps_stations()

Found 29 HOLAPS stations: ['BE-Bra', 'BE-Vie', 'CH-Dav', 'CZ-BK1', 'CZ-wet', 'DE-Geb', 'DE-Gri', 'DE-HoH', 'DE-Hzd', 'DE-Kli', 'DE-RuR', 'DE-RuS', 'DE-Tha', 'DK-Sor', 'FI-Hyy', 'FI-Let', 'FR-Aur', 'FR-Bil', 'FR-Hes', 'IT-BCi', 'IT-Cp2', 'IT-MBo', 'IT-Ren', 'IT-SR2', 'IT-Tor', 'SE-Deg', 'SE-Htm', 'SE-Nor', 'SE-Svb']

Processing HOLAPS station: BE-Bra
Loading HOLAPS data for BE-Bra...
Data period: 2001 to 2020
Total years: 20
HOLAPS variable availability saved to c:\\Deepak\\HOLAPS\\monthly\\station_data\BE-Bra\BE-Bra_holaps_variable_availability.csv
Performing yearly analysis...
Yearly means saved to c:\\Deepak\\HOLAPS\\monthly\\station_data\BE-Bra\holaps_yearly_means_BE-Bra.xlsx
Performing seasonal analysis...
Seasonal means for DJF saved to c:\\Deepak\\HOLAPS\\monthly\\station_data\BE-Bra\holaps_seasonal_means_BE-Bra_DJF.xlsx
Seasonal means for MAM saved to c:\\Deepak\\HOLAPS\\monthly\\station_data\BE-Bra\holaps_seasonal_means_BE-Bra_MAM.xlsx
Seasonal means for JJA saved to c:\\Deepak

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.lines import Line2D

# Define HOLAPS parameters and assign fixed colors
color_dict = {
    'GHF': '#1f77b4',          # blue
    'H': '#ff7f0e',            # orange
    'LE': '#2ca02c',           # green
    'Rn': '#d62728',           # red
    'Ta': '#9467bd',           # purple
    'Td': '#8c564b',           # brown
    'Ts': '#e377c2',           # pink
    'rain': '#17becf',         # teal
    'sm1': '#bcbd22'           # olive
}

parameters = list(color_dict.keys())

# Fixed absolute Y positions for each parameter (distance from zero line)
# Increased distances for better separation
parameter_distances = {
    'GHF': 0.2,
    'H': 0.4,
    'LE': 0.6,
    'Rn': 0.8,
    'Ta': 1.0,
    'Td': 1.2,
    'Ts': 1.4,
    'rain': 1.6,
    'sm1': 1.8
}

# Directory containing Excel files
input_directory = r'C:\Deepak\HOLAPS\monthly\station_data'
output_directory = r'C:\Deepak\HOLAPS\monthly\station_data\plots'

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Get list of all trend analysis Excel files
trend_files = [f for f in os.listdir(input_directory) if f.endswith('_yearly.xlsx') and 'trend' in f.lower()]

# Process each station file
for trend_file in trend_files:
    try:
        # Extract station name from filename
        station_name = trend_file.replace('holaps_trend_tables_', '').replace('_yearly.xlsx', '')
        
        # Create figure
        fig, ax = plt.subplots(1, 1, figsize=(20, 12))
        
        # File path
        file_path = os.path.join(input_directory, trend_file)
        
        # Read the data
        p_values_df = pd.read_excel(file_path, sheet_name='P-values')
        slopes_df = pd.read_excel(file_path, sheet_name='Slopes')
        
        # Clean the data
        p_values_df = p_values_df.dropna(how='all')
        slopes_df = slopes_df.dropna(how='all')
        p_values_df = p_values_df.reset_index(drop=True)
        slopes_df = slopes_df.reset_index(drop=True)
        
        # For HOLAPS, we typically have one row per station (Complete Period)
        periods = slopes_df['Period'].values
        analyses = slopes_df['Analysis'].values
        
        # Plot data for each period/analysis
        for period_idx, (period, analysis) in enumerate(zip(periods, analyses)):
            for param in parameters:
                if param not in slopes_df.columns or param not in p_values_df.columns:
                    continue
                    
                slope = slopes_df[param].iloc[period_idx]
                p_val = p_values_df[param].iloc[period_idx]

                if pd.isna(p_val) or pd.isna(slope):
                    continue

                # FIXED POSITION: Use the predefined distance from zero
                # If slope is positive: above zero (positive Y)
                # If slope is negative: below zero (negative Y)
                base_distance = parameter_distances[param]
                y_position = base_distance if slope >= 0 else -base_distance
                
                x_position = period_idx  # Center on the period position
                
                # Small random offset to avoid perfect alignment
                x_offset = np.random.uniform(-0.15, 0.15)
                x_position += x_offset

                # Marker rules based on p-values - INCREASED SIZES
                if p_val < 0.1:
                    marker = 'o'   # full circle
                    marker_size = 300
                    fill_style = 'full'
                    edge_width = 4.0
                    alpha_val = 1.0
                elif p_val < 0.2:
                    marker = '^'   # triangle
                    marker_size = 260
                    fill_style = 'full'
                    edge_width = 4.0
                    alpha_val = 0.9
                else:
                    marker = 'o'   # open circle
                    marker_size = 220
                    fill_style = 'none'
                    edge_width = 3.5
                    alpha_val = 0.8

                # Plot marker
                ax.scatter(x_position, y_position, s=marker_size,
                           c=color_dict[param] if fill_style == 'full' else 'none',
                           marker=marker, edgecolors=color_dict[param],
                           linewidths=edge_width, alpha=alpha_val, zorder=5)
        
        # Customize plot - INCREASED FONT SIZES
        ax.set_xlabel('Analysis Periods', fontsize=20, fontweight='bold')
        ax.set_xticks(range(len(periods)))
        
        # Create labels showing both analysis type and period
        x_labels = [f"{analyses[i]}\n({periods[i]})" for i in range(len(periods))]
        ax.set_xticklabels(x_labels, rotation=45, ha='right', fontsize=16)
        
        # Set y-axis limits based on maximum distance
        max_distance = max(parameter_distances.values())
        y_limit = max_distance * 1.2  # Add some padding
        ax.set_ylim(-y_limit, y_limit)
        
        # Remove y-axis labels and ticks as in the original code
        ax.set_yticks([])
        ax.set_ylabel('')
        
        # Zero line - INCREASED THICKNESS
        ax.axhline(y=0, color='black', linestyle='-', alpha=0.8, linewidth=3, zorder=1)
        
        # Customize spines
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_visible(False)

        # Add vertical lines to separate analysis groups - INCREASED VISIBILITY
        for i in range(len(periods) + 1):
            ax.axvline(x=i - 0.5, color='gray', linestyle='--', alpha=0.5, linewidth=2, zorder=1)

        # Add grid for better readability - INCREASED VISIBILITY
        ax.grid(True, axis='y', linestyle='--', alpha=0.4, linewidth=1.2, zorder=1)

        # Main title - INCREASED SIZE
        plt.title(f'HOLAPS Trend Analysis for {station_name}',
                 fontsize=22, fontweight='bold', pad=20)

        # Create legends - INCREASED SIZES
        legend_elements = [
            Line2D([0], [0], marker='s', color='w',
                   markerfacecolor=color_dict[param],
                   markersize=16, label=param)
            for param in parameters
        ]

        marker_elements = [
            Line2D([0], [0], marker='o', color='black', markerfacecolor='black',
                   markersize=20, linestyle='None', label='p < 0.1 (Full circle)'),
            Line2D([0], [0], marker='^', color='black', markerfacecolor='black',
                   markersize=20, linestyle='None', label='0.1 ≤ p < 0.2 (Triangle)'),
            Line2D([0], [0], marker='o', color='black', markerfacecolor='none',
                   markersize=20, linestyle='None', label='p ≥ 0.2 (Open circle)')
        ]

        # Place legends - INCREASED FONT SIZES
        legend1 = ax.legend(handles=legend_elements, 
                          loc='upper left',
                          bbox_to_anchor=(1.02, 1),
                          fontsize=14,
                          title='HOLAPS Parameters', 
                          title_fontsize=16)

        legend2 = ax.legend(handles=marker_elements, 
                          loc='upper left',
                          bbox_to_anchor=(1.02, 0.7),
                          fontsize=14,
                          title='Significance Levels', 
                          title_fontsize=16)

        # Add both legends to the plot
        ax.add_artist(legend1)
        
        # Adjust layout for larger elements
        plt.tight_layout(rect=[0, 0, 0.82, 0.95])
        
        # Save the plot with high DPI
        output_path = os.path.join(output_directory, f"{station_name}_holaps_trend_plot.png")
        plt.savefig(output_path, dpi=350, bbox_inches='tight')
        plt.close()
        
        print(f"HOLAPS trend plot saved for {station_name}")
        
    except Exception as e:
        print(f"Error processing {trend_file}: {str(e)}")
        import traceback
        print(traceback.format_exc())

print("All HOLAPS files processed!")

All HOLAPS files processed!
